In [2]:
import pandas as pd
import numpy
from timeit import default_timer as timer
from models import Transaction, Item, Transaction_Item

In [3]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}

In [4]:
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [5]:
chunk_iterator = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   chunksize=20000,
                   nrows=100000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [6]:
def extract_transaction(index, row):
    return (
        ('id', index),
        ('time', row['資料日期與時間']),
        ('time_phase', row['餐別帶']),
        ('branch_id', row['店舖代號']),
        ('location_type', row['主商圈']),
        ('location', row['縣市別'])
    )

def extract_item(row):
    return (
        ('type', row['品號-品名稱']),
        ('subtype', row['群號-群名稱']),
        ('name', row['單品名稱']),
        ('price', row['銷售單價'])
    )

def extract_transaction_item(index, row):
    return (
        ('transaction_id', index),
        ('item_name', row['單品名稱']),
        ('times', row['銷售數量']),
        ('transaction_amount', row['交易金額'])
    )

def write(df):
    items_added = 0
    transactions_added = 0
    transaction_record_added = 0
    item_set = set()
    transaction_set = []
    transaction_record_set = []

    for index, row in df.iterrows():
        item_set.add(extract_item(row))
        transaction_set.append(extract_transaction(index, row))
        transaction_record_set.append(extract_transaction_item(index, row))

    for item in item_set:
        is_added = Item.create(**dict(item))
        if is_added:
            items_added += 1

    for transaction in transaction_set:
        is_added = Transaction.create(**dict(transaction))
        if is_added:
            transactions_added += 1
    
    for record in transaction_record_set:
        record = dict(record)
        item = Item.get(name=record['item_name'])
        if item is None:
            continue
        record['item_id'] = item.id
        del record['item_name']
        is_added = Transaction_Item.create(**record)
        if is_added:
            transaction_record_added += 1
    print('\rItems added: {}\nTransactions added: {}\nRecords added: {}\n'.format(items_added, transactions_added, transaction_record_added), flush=True)

In [7]:
start = timer()
for chunk in chunk_iterator:
    chunk = chunk.dropna()
    write(chunk)
end = timer()
print('Total time elapsed: {}'.format(end - start))

Items added: 0
Transactions added: 0
Records added: 0

Items added: 0
Transactions added: 0
Records added: 0

Items added: 0
Transactions added: 0
Records added: 0

Items added: 0
Transactions added: 0
Records added: 0

Items added: 0
Transactions added: 0
Records added: 0

Total time elapsed: 365.2618576060049


In [15]:
from models import engine
query = """
SELECT * FROM transaction_item
JOIN transactions ON transaction_item.transaction_id = transactions.id
JOIN items ON items.id = transaction_item.item_id;
"""
records = pd.read_sql_query(query, index_col=['transaction_id'], con=engine)

In [16]:
records

,item_id,times,transaction_amount,id,time,time_phase,location,branch_id,location_type,id,type,subtype,name,price
transaction_id,,,,,,,,,,,,,,
01564720171201180924000113751602,150,2,49.0,01564720171201180924000113751602,2017-12-01 18:09:24,晚餐時間帶,台中市,3860,住宅型,150,34-健康飲料,343-水,多喝水２Ｌ,35.0
01666520171201100642000115757802,326,1,0.0,01666520171201100642000115757802,2017-12-01 10:06:42,一般時間帶,台中市,3477,車站型,326,00-新型態代收,15-E商品,日翊退貨通,0.0
01564720171201230020000113771102,265,1,25.0,01564720171201230020000113771102,2017-12-01 23:00:20,一般時間帶,台中市,3860,住宅型,265,35-口香糖,352-無糖口香糖,ｓｕｐｅｒ冰炫薄荷口香糖,39.0
01475120171201082535000111653902,87,1,13.0,01475120171201082535000111653902,2017-12-01 08:25:35,早餐時間帶,新竹市,4151,工業型,87,03-蒸箱食品,031-一般包子,芋泥包,15.0
01475120171201185612000118817601,378,1,76.0,01475120171201185612000118817601,2017-12-01 18:56:12,晚餐時間帶,新竹市,4151,工業型,378,12-調理麵,124-義大利麵,多重起司烤雞焗烤麵,89.0
01564720171201144614000116434001,188,1,20.0,01564720171201144614000116434001,2017-12-01 14:46:14,午餐時間帶,台中市,3860,住宅型,188,29-冷藏飲料,291-冷藏茶飲料,蘋果冰茶,25.0
01564720171201153901000116436501,62,1,35.0,01564720171201153901000116436501,2017-12-01 15:39:01,下午茶時間帶,台中市,3860,住宅型,62,15-小菜、微波,155-微波小吃,香辣霸王雞球,35.0
01666520171201222854000110631501,50,1,32.0,01666520171201222854000110631501,2017-12-01 22:28:54,一般時間帶,台中市,3477,車站型,50,73-寵物、園藝,732-貓食,偉嘉海洋大餐,32.0
01614720171201210842000114417702,177,1,20.0,01614720171201210842000114417702,2017-12-01 21:08:42,一般時間帶,台中市,10697,住宅型,177,31-茶飲料,312-綠茶,茶裏王日式綠茶,20.0


In [ ]:
台中
台中跟新竹
早餐時間
晚餐時間
早餐跟晚餐
台中跟晚餐